# Metalanguage functions

Functions are the bread and butter of the typed metalanguage. There are two main ways to describe a function: via a lambda expression, and via a map (e.g. a data structure similar to a python `dict`). There is also a construct that allows combining functions, with similar behavior to the python library's `ChainMap` class.

All classes implementing a functional type provide a standard interface for application and reduction. Key API elements for general use:

1. "Calling" a function via the standard python interface produces a composite expression representing the function combined with an argument. This will of course fail if the types are not compatible.
2. The member function `apply` will take an argument and produce a beta-reduced expression given that argument, without leaving a derivation. (This is nearly equivalent to call + reduce, with the derivation being the primary difference in outcome.)

A class that can have a functional type should also implement:

* The function `will_reduce` takes an argument, and returns `True` just in case a composite function-argument expression consisting of the function with the supplied argument would succeed in reducing. (For lambda expressions, this returns only `True`, but for more complex function types, there are conditions under which it may fail.)

Finally, some functional expressions may raise a python `DomainError` exception if they have some non-trivial domain restriction, and are called with an argument that is outside of this domain.

## Lambda expressions

A metalanguage lambda expression corresponds closely to the standard typed lambda expressions that are familiar from linguistic theory, and also functional programming. In this section I will assume some familiarity with the formal properties of a lambda expression.

A lambda expression consists of the lambda operator, a typed variable name, and a body. In the following example the body is the expression `P_<e,t>(x)`. The variable `x` is bound in this subexpression and has a type determined by the binder.

In [ ]:
%te L x_e: P_<e,t>(x)

If instances of the bound variable and the binder type can't be unified, then parsing will fail; if they can be unified then all instances of the variable will be adjusted to the principal type. For example, in the resulting expression, the property `P` forces `x` to be of type `e`, which percolates to the binder.

In [ ]:
%te L x_X: P_<e,t>(x)

Lambda expressions can of course be embedded in other lambda expressions, and support application and reduction as expected.

In [ ]:
e = %te (L x_X: P_<e,t>(x))(_c1)
e.reduce_all().derivation

Variable names of bound variables are not guaranteed to stay the same on lambda application.  Variable names of free variables are safe.  In the following example, `x` is free in the main argument in the definition of `e7`.  On application, the bound `x` must be renamed to avoid collision with the free `x`.

In [ ]:
%te reduce (L p_t : L x_e : p & P_<e,t>(x))(P_<e,t>(x))

In [ ]:
# another example of a collision where both variables are bound.  The one that is locally bound gets renamed.
%te reduce L x_e : (L p_t : L x_e : p & P_<e,t>(x))(P_<e,t>(x))

## Maps

Functions can also be implemented directly as a mapping, implemented via the class `lamb.core.MapFun`. The metalanguage parser accepts a relatively standard python `dict`-like syntax.

In [ ]:
f = %te {3:x_n, 4:5}
display(f, f.type)

Maps are inherently partial relative to the domain implied by their type. The above example is a function from numbers to numbers, but is only defined for two numbers. If a number outside the domain is provided as an argument, reduction will raise a `DomainError` (this does not happen on application).

In [ ]:
with lamb.errors():
    %te simplify {3:x_n, 4:5}(5)

A map involves unique domain elements: every domain element is present at most once in the map. However, the metalanguage follows standard python syntax in allowing maps to be written with duplicates. Order is used to resolve duplicates in the domain. For example, the following example is syntactically valid but essentially ignores the first pair with a `3`.

In [ ]:
%te {3:2, 3:x_n, 4:5}

Note that maps in this syntax that consist entirely of domain elements will be translated to `MetaTerm` expressions (see below) rather than `MapFun` elements (although the class does support entirely concrete maps). A map that has non-domain-element terms (variables or unvalued constant terms) anywhere in it is in a sense incomplete: it describes a function whose outputs can't be fully determined without more information. If the terms are in the range of the function, reduction can proceed, but may of course return a non-valued term.

In [ ]:
%te simplify {3:x_n, 4:5}(3)

**The empty function**. In contrast to ordinary python, `{}` does not give an empty map function, but rather an empty set. To instantiate an empty function, you can use the syntax `Fun()`:

In [ ]:
te("Fun()")

In [ ]:
%te Fun()

Similar to an unconstrained empty set, the empty function has a polymorphic type, indicating that its domain and range could be anything. In context, this will typically become more concrete.

In [ ]:
%te Fun() + {1:_c1}

The `Fun` syntax can also be used to produce functions from sets of pairs, following a standard set-theoretic treatment of functions. (Note that duplicate ordering for this instantiation method respects set order normalization, which uses lexicographic ordering by element; it therefore can be less predictable than writing a map directly.)

In [ ]:
%te Fun({(1,2), (3,4)})

**Non-concrete domains**. If a non-valued term is in the domain of the map, then things become more complicated than if the domain is fully concrete. Consider the following map:

In [ ]:
f = %te {4:5, x_n:3}
f

This function, like all maps, is only defined for some domain elements, but it is not yet determined what those elements are. In fact, it may well be that `x` resolves to `4`, in which case there's a duplicate domain element. If this happens, the ordering rule is used to resolve duplicate domain elements on simplification. For this reason, if a map has unvalued domain elements, reduction can't proceed at all until they are resolved. The following derivation illustrates this; reduction of the lambda expression generates a map with duplicate domain elements, and the first one is eliminated by simplification.

In [ ]:
x = %te simplify (L x_n : {4:5, x_n:3})(4)
x.derivation

Of course this sort of "shadowing" can work the other way, if the order of the map is different.

In [ ]:
x = %te simplify (L x_n : {x_n:3,4:5})(4)
x.derivation

Because a variable in a map may be shadowed, it is also not safe to assume that reduction with the variable name is safe in the general case. For example, `{x_n:3,4:5}(x_n)` will resolve to 5 if `x` is 4, and to `3` otherwise; we can't know which without knowing `x`. (There are special cases that could be handled, but that simplify does not curently deal with.)

(This example also shows a case of a derivation leading to duplicate domain elements; when writing a map with this property directly in the metalanguage syntax, no intermediate representation with these duplicates is generated, so such cases may be a bit rare.)

**Programmatic use**. The `MapFun` class has a somewhat complex internal representation involving a sequence of pairs together with a mapping object. To programmatically instantiate one of these objects, you may want to use the class method `from_dict` rather than the standard constructor, which takes a sequence of Tuples.

In [ ]:
x = meta.core.MapFun.from_dict({te(1):te(2), te(3):te(4)})
x

In [ ]:
set(x) # show the pairs

When instantiating a `MapFun` programmatically, keep in mind that domain duplicates aren't resolved until simplification. Reduction on a structure with domain duplicates is well-defined and is equivalent to simplify+reduce.

## `MetaTerm` functions

See the documentation notebook *Domain elements and `MetaTerm`s for the primary documentation on `MetaTerm` functions. In brief, a `MetaTerm` with a functional type can be backed by:

* A `frozendict` mapping from concrete domain elements to domain elements. This construct is very similar to `MapFun` functions, as described above.
* A `frozenset`, for characteristic functions corresponding to that set (i.e. functions of type `<X,t>` for some `X`).
* A pure python function, in which case all responsibility for type safety (etc) devolves to that function.

## Chained functions

Functions can be combined using the `+` operator (corresponding to the class `meta.core.ChainFun`), which has similar behavior to the standard library [`collections.ChainMap`](https://docs.python.org/3/library/collections.html#collections.ChainMap) class. Reduction on a chained function will proceed from right to left among the combined functions, resolving duplicates in that order. If the combined functions are maps, they can always be simplified, though chains involving lambda expressions cannot be completely simplified. Application/reduction does not require a simplifiable chain.

In [ ]:
%te simplify {1:2, 3:4} + {3:5}

In [ ]:
%te simplify {1:2, 3:4} + {x_n:5}

In [ ]:
# total function on numbers that returns a special-cased value for `3`
%te simplify ((L x_n : 0) + {3:5})(3)

A note for python programmers used to the quirks of python `update`: chaining a function involving duplicates changes the key order in the simplified version, in contrast to the behavior of the `update` function. For example, in the following case, the chain overwrites the original `1:2` pair (regular `update` would also do this), but additionally changes key order so that `1` follows `3`. For functions with fully concrete domains this is cosmetic, but it can interact with variable resolution in useful ways.

In [ ]:
%te simplify {1:2, 3:4} + {1:5}

One way of thinking about this is that we want to guarantee that reduction on an unsimplified chain is equivalent to reduction on a simplified chain, regardless of the sequencing of variable valuation. For example, in the following case, if `x` resolves to `3`, then the right side of the chain should override the `3:4` pair, and the standard `update` behavior wouldn't accomplish this.

In [ ]:
x = %te simplify (L x_n : {x_n:2, 3:4} + {x_n:5})(3)
x.derivation

## Function properties

The metalanguage supports accessing domain and codomain sets via the `Dom` and `Codom` operators respectively. Given some function `f` of type `<X,Y>`, `Dom(f)` will have type `{X}`, and `Codom(f)` will have type `{Y}`. For example, here is the domain set for a simple lambda expression:

In [ ]:
%te simplify Dom(L x_e: True)

The range and codomain of a function are sometimes conflated, but when they are distinguished, the range is the set of actual values used, whereas the codomain is the set of possible values used. Only the latter is computable in the general case. For the case of map functions in the metalanguage, the two coincide, but otherwise, the metalanguage makes no attempt to determine the set of actual range elements. The constant function above illustrates a case where the range ( which would just be `{True}`) and the codomain come apart:

In [ ]:
%te simplify Codom(L x_e: True)

Using these properties with map-based functions is straightforward, but other cases have a few caveats.

* Map-based functions have their domain/codomain sets drawn directly from the keys and values of the map.
* Lambda function domain/codomain produce potentially non-finite sets (e.g. sets like `{x_X: True}`), and inherit all the caveats of such sets. Currently, these functions are *not* sensitive to definedness conditions.
* Chained functions have domains/codomain that are simply the union of the parts, and so therefore may be non-finite.
* The domain and codomain of unvalued terms can't be determined, and so simplification is necessarily deferred.
* Some cases are not handled in compilation, and it may be helpful to simplify first.

Examples of map-based (co)domains illustrating simplification:

In [ ]:
te("Dom({1:2, 3:4, 5:4})").simplify_all().derivation

In [ ]:
te("Codom({1:2, 3:4, 5:4})").simplify_all().derivation

In [ ]:
te("Dom({1:2, 3:4, x_n:4})").simplify_all().derivation

Examples of lambda expression (co)-domains:

In [ ]:
te("Dom(L x_e: P_<e,t>(x))").simplify_all().derivation

In [ ]:
te("Codom(L x_e: P_<e,t>(x))").simplify_all().derivation

Examples of chained (co)-domains:

In [ ]:
te("Dom({1:2, 3:x_n} + ({4:5}))").simplify_all().derivation

In [ ]:
te("Codom({1:2, 3:x_n} + ({4:5}))").simplify_all().derivation

In [ ]:
te("Dom({1:2, 3:x_n} + (L x_n : 2))").simplify_all().derivation

In [ ]:
te("Codom({1:2, 3:x_n} + (L x_n : 2))").simplify_all().derivation

A few examples of compilation. Domain restrictions are necessary to handle non-finite domains, as per usual:

In [ ]:
with types.type_n.restrict_domain(values=[1, 2, 3]):
    display(lamb.meta.exec(te("Dom(L x_n: 1)")))

In [ ]:
with types.type_n.restrict_domain(values=[1, 2, 3]):
    display(lamb.meta.exec(te("Codom(L x_n: 1)")))

In [ ]:
with types.type_n.restrict_domain(values=[1, 2, 3]):
    display(lamb.meta.exec(te("Dom((L x_n: 1) + {3:1})")))

In [ ]:
lamb.meta.exec(te("Codom({1:2} + {3:1})"))

In [ ]:
lamb.meta.exec(te("Codom(L x_e: True)"))

## Example: assignment functions in the metalanguage

The following example illustrates a practical application of map functions: they can be used to manipulate metalanguage assignment functions. The sketch is pretty minimal but it shows how to do pronominal reference, binding, and assignment-sensitive function application. (A more sophisticated example might illustrate the full reader monad for assignments, along the lines of Shan 2002, *Monads for natural language semantics*. This example leaves off the unit operation as well as the derivation of the lifted FA rule.)

In [ ]:
reload_lamb()

In [ ]:
%%lamb
||Trace|| =  L i_n: L f_<n,e>: Partial(f(i), i << Dom(f))
||_1|| = 1
||Binder|| = L i_n: L g_<<n,e>,X>: L f_<n,e>: L x_e : g(f + {i:x})
||barked|| = L f_<n,e>: L x_e: Barked_<e,t>(x)
||Closure|| = L f_<<n,e>,X> : f(Fun()) # Convert assignment types down to ordinary denotations

Here we are using a `Partial` object to guard appplication for a potentially partial `f`. If this domain condition is not met, the derivation will result in an undefined `Partial`:

In [ ]:
Closure * (Trace * _1)

Assignment-sensitive FA, given a function and an argument that also take an assignment parameter, will combine the function and the argument, passing the assignment through. (Again, see Shan 2002 for the full details.)

In [ ]:
assign_FA = %te L fun_<<n,e>,<X,Y>> : L x_<<n,e>,X> : L f_<n,e> : fun(f)(x(f))
lang.get_system().add_binary_rule(assign_FA, 'AFA')

A trace, given some assignment, returns the value of the assignment at its index.

In [ ]:
(Trace * _1)

When composing this trace with the intransitive verb `barked`, the AFA rule will kick in:

In [ ]:
((Trace * _1) * barked)[0].content.calculate_partiality()

We can build a meaning for something like a relative clause `that barked t_1` by adding in a binder, which shifts the assignment function using a chained function with a map (i.e. the subformula `f + {1:x_e}`, where `x_e` is bound by a lambda operator).

In [ ]:
(Binder * _1) * ((Trace * _1) * barked)

The closure operator in this example saturates an outer assignment parameter with the empty function, returning the ordinary meaning for the structure:

In [ ]:
x = ((Binder * _1) * ((Trace * _1) * barked))[0].content
x

In [ ]:
Closure * ((Binder * _1) * ((Trace * _1) * barked))

In [ ]:
(Closure * ((Binder * _1) * ((Trace * _1) * barked))).tree()

In [ ]:
(Closure * ((Binder * _1) * ((Trace * _1) * barked)))[0].content.derivation.trace()